In [1]:
import datafaucet as dfc

In [2]:
# start the engine
engine = dfc.engine('spark')

created SparkEngine
Init engine "spark"
Connecting to spark master: local[*]
Engine context spark:2.4.4 successfully started


In [3]:
df = dfc.range(1000)

In [4]:
from pyspark.sql import functions as F

df = (df
    .cols.create('prefix').randchoice(['032', '033','035', '095', '093', '098'])  
    .cols.create('number').randint(1111111, 9999999)  
    .cols.get('number').cast('string')
    .cols.create('f').fake('first_name')
    .cols.create('l').fake('last_name')
).cache()

#concat trunc + code + number
df = df.select(F.concat('prefix', 'number').alias('number'), 'f', 'l')

In [5]:
df.count()

1000

In [6]:
df.data.grid(5)

,number,f,l
0,0326027256,Angela,Green
1,0353339102,John,Cox
2,0322970097,Lindsay,Allen
3,0327475254,Rachel,Young
4,0329657164,Deanna,Collins


In [9]:
from pyspark.sql import functions as F
from datafaucet.spark.functions import obscure

def mobile_obscure(key):
    return lambda c: F.concat(F.substring(c, 2, 3), F.lit('%'), obscure(key)(c))
    
res = df.cols.get('number').apply(mobile_obscure('mysecret'))
res.data.grid(5)

,number,f,l
0,326%i/VyDA52CHKL8AcA,Angela,Green
1,353%i/VyCwtwjHGN9QYA,John,Cox
2,322%i/VyDI9yDXUJ8QMA,Lindsay,Allen
3,327%i/VyDAp3DXCL8AUA,Rachel,Young
4,329%i/VyjAl1D3KN9gUA,Deanna,Collins


In [10]:
from pyspark.sql import functions as F
from datafaucet.spark.functions import unravel

def mobile_unravel(key):
    return lambda c: unravel(key)(F.element_at(F.split(c, '%'),2))

res.cols.get('number').apply(mobile_unravel('mysecret')).data.grid(5)

,number,f,l
0,0326027256,Angela,Green
1,0353339102,John,Cox
2,0322970097,Lindsay,Allen
3,0327475254,Rachel,Young
4,0329657164,Deanna,Collins


In [11]:
res.cols.get('number').apply(mobile_unravel('wrongsecret')).data.grid(5)

,number,f,l
0,*8.=437%**,Angela,Green
1,*8)8729&/.,John,Cox
2,*8.9=60'&+,Lindsay,Allen
3,*8.<065%*(,Rachel,Young
4,*8.2247&)(,Deanna,Collins
